## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the "pax" column, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at [neobase in Github](https://github.com/alexprengere/neobase))

Bonus point: Solve this problem using pandas (instead of any other approach)


Suggestion: follow the below plan of action:

* Get familiar with the data
* Select columns of interest
* Decide what to do with NaNs

* Make processing plan
* Develop code that works with a sample

* Adjust the code to work with Big data
* Test big data approach on a sample

* Run program with big data


## 1) Get familiar with data

In [2]:
import pandas as pd
import numpy as np

In [4]:
bookings_file = 'bookings.csv.bz2'
searches_file = 'searches.csv.bz2'

### What if we don't want to read the whole file?

Options:

* prepare the sample

* read_csv with nrows option

In [22]:
# No hace falta descomprimir primero (salvo en el caso de .zip)

sample = pd.read_csv(bookings_file, nrows = 100000, sep = '^')

sample.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [6]:
# Si quiero que salgan todas las columnas:

pd.options.display.max_columns = None

sample.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [8]:
sample.shape

(100000, 38)

In [9]:
sample.describe()

,duration,distance,intl,pax,year,month
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000
mean,22960.511580,476.904440,0.662550,0.509630,2013.0,2.191040
std,46979.388683,1780.890791,0.472842,2.219013,0.0,1.260222
min,51.000000,0.000000,0.000000,-80.000000,2013.0,1.000000
25%,3370.750000,0.000000,0.000000,-1.000000,2013.0,1.000000
50%,8594.000000,0.000000,1.000000,1.000000,2013.0,3.000000
75%,21535.500000,0.000000,1.000000,1.000000,2013.0,3.000000
max,791131.000000,19012.000000,1.000000,70.000000,2013.0,5.000000


In [11]:
sample.info()  # Nos da, entre otras cosas, el memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 38 columns):
act_date               100000 non-null object
source                 100000 non-null object
pos_ctry               100000 non-null object
pos_iata               100000 non-null object
pos_oid                100000 non-null object
rloc                   100000 non-null object
cre_date               100000 non-null object
duration               100000 non-null int64
distance               100000 non-null int64
dep_port               100000 non-null object
dep_city               100000 non-null object
dep_ctry               100000 non-null object
arr_port               100000 non-null object
arr_city               100000 non-null object
arr_ctry               100000 non-null object
lst_port               100000 non-null object
lst_city               100000 non-null object
lst_ctry               100000 non-null object
brd_port               100000 non-null object
brd_city          

Clean the column names

In [12]:
sample.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [23]:
sample.columns = sample.columns.str.strip()
sample.columns

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

## 2) Select the columns of interest 

In [24]:
sample = sample[['arr_port', 'pax', 'year']]
sample.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In the sample everything might be ok, but we should prepare for NaN case

In [26]:
sample.count()  # No hay NaNs

arr_port    100000
pax         100000
year        100000
dtype: int64

## 4) Make processing plan
1) get only the bookings from 2013

2) group by arr_port, sum

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [27]:
sample_2013 = sample[sample['year'] == 2013]
sample_2013.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


#### 4.2) group by arr_port, sum

In [33]:
paxes_per_airport = sample_2013.groupby(by = 'arr_port')['pax'].sum()

paxes_per_airport.head()

arr_port
AAE          2
AAL         16
AAQ          5
AAR          4
ABE         16
Name: pax, dtype: int64

In [38]:
paxes_per_airport.sort_values(ascending = False).head(10)

arr_port
LHR         1006
MCO          861
JFK          795
LAX          761
BKK          747
LAS          732
SFO          705
ORD          686
CDG          676
DXB          587
Name: pax, dtype: int64

Esto es el top10 para la muestra de 1000

## 5) Adjust the code to work with Big data


Hint: check out https://pandas.pydata.org/pandas-docs/stable/io.html#io-chunking

Los chunks funcionan de la siguiente manera: al declarar chunksize, tenemos un objeto Reader en vez de un dataframe que es lazy -> va teniendo en memoria sólo el chunk

In [45]:
sample = pd.read_csv(bookings_file, nrows = 100000, sep = '^', chunksize = 10000)

sample

In [46]:
# Los chunks son df

for chunk in sample:
    print(type(chunk))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [50]:
partial_results = []

for chunk in sample:
    
    chunk = chunk[['arr_port', 'pax', 'year']].dropna()
    chunk = chunk[sample['year'] == 2013]
    result_from_this_chunk = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(result_from_this_chunk)

In [52]:
partial_results

[]

Esto está vacío porque ya hemos leído el sample con la celda del print. Como el sample es un Reader, al hacer eso se ha quedado en el final.

In [58]:
sample = pd.read_csv(bookings_file, nrows = 100000, sep = '^', chunksize = 10000)

partial_results = []

for chunk in sample:
    
    chunk = chunk[['arr_port', 'pax', 'year']].dropna()
    chunk = chunk[chunk['year'] == 2013]
    result_from_this_chunk = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(result_from_this_chunk)
    
partial_results[1].head()

arr_port
AAL         3
ABJ         3
ABQ         6
ABV         7
ABY         1
Name: pax, dtype: int64

#### Now we need to put together the results from all the chunks

Options:

* df.append()

* pd.concat()
    

Para juntarlos, en vez de un bucle de joins, pd.concat

In [60]:
pd.concat(partial_results).head()

arr_port
AAL          0
ABQ         12
ABV          2
ABZ         -6
ACC          4
Name: pax, dtype: int64

Agrupamos

In [62]:
pd.concat(partial_results).groupby('arr_port').sum().head()

arr_port
AAE          2
AAL         16
AAQ          5
AAR          4
ABE         16
Name: pax, dtype: int64

Ordenamos

In [63]:
pd.concat(partial_results).groupby('arr_port').sum().sort_values(ascending = False).head(10)

arr_port
LHR         1006
MCO          861
JFK          795
LAX          761
BKK          747
LAS          732
SFO          705
ORD          686
CDG          676
DXB          587
Name: pax, dtype: int64

### We have to read the whole file, but with nrows we are reading always the first N rows


## Complete Solution

In [64]:
from datetime import datetime

Le metemos un controlador de tiempo

In [ ]:
%%time

data = pd.read_csv(bookings_file, sep = '^', chunksize = 1000000)

partial_results = []

for chunk_num, chunk in enumerate(data):
    
    print('Start processing chunk %d at %s' % (chunk_num, datetime.now()))
    
    chunk = chunk[['arr_port', 'pax', 'year']].dropna()
    chunk = chunk[chunk['year'] == 2013]
    result_from_this_chunk = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(result_from_this_chunk)
    
all_results = pd.concat(partial_results)
total_arrivals = all_results.groupby('arr_port').sum()
top_10 = total_arrivals.sort_values(ascending = False).head(10)

Start processing chunk 0 at 2018-06-01 19:01:25.794888
Start processing chunk 1 at 2018-06-01 19:01:49.294829
Start processing chunk 2 at 2018-06-01 19:02:13.577551
Start processing chunk 3 at 2018-06-01 19:03:00.838490
Start processing chunk 4 at 2018-06-01 19:04:02.649223


<string>:2: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.


Start processing chunk 5 at 2018-06-01 19:06:13.380735


In [ ]:
top_10

Solución mucho más fácil:

In [6]:
%%time

df = pd.read_csv(bookings_file, sep = '^', usecols = ['arr_port', 'pax', 'year'])

res = df[df['year'] == 2013].groupby('arr_port')['pax'].sum().sort_values(ascending = False).head(10)



CPU times: user 2min 53s, sys: 1.11 s, total: 2min 55s
Wall time: 2min 55s


In [7]:
res

arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
Name: pax, dtype: float64

Para ver el espacio en memoria que utiliza un objeto en python

In [8]:
import sys

sys.getsizeof(df)  # Cuidado porque con listas a veces no es del todo fiable. Con df sí.

# Lo que me de entre 10^6 es en MB.

810000914

### Bonus point

Nombre de los aeropuertos:

De la base de datos de `geobases` podemos instalar el paquete e importarlo en python

In [9]:
! pip install neobase

    100% |████████████████████████████████| 3.1MB 401kB/s eta 0:00:01
  Running setup.py bdist_wheel for neobase ... done
  Stored in directory: /home/dsc/.cache/pip/wheels/c0/e1/42/f8c7ecf51ff5d3ea224b9af47f98b09ca03c3decb63e44e727
Successfully built neobase
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import neobase

nb = neobase.NeoBase()  # Creo un objeto NeoBase y cuando haga nb. me dejará hacer cosas

In [11]:
nb.get('JFK')

{'__dup__': set(),
 'city_code_list': ['NYC'],
 'city_name_list': ['New York City'],
 'continent_name': 'North America',
 'country_code': 'US',
 'iata_code': 'JFK',
 'lat': '40.63983',
 'lng': '-73.77874',
 'location_type': ['A'],
 'name': 'John F. Kennedy International Airport',
 'timezone': 'America/New_York'}

In [13]:
type(nb.get('JFK'))  # Es un diccionario

dict

In [14]:
nb.get('JFK')['name']

'John F. Kennedy International Airport'

In [ ]:
res.index = res.index.map(lambda airport: nb.get(airport.strip())['name'])

In [27]:
res

arr_port
London Heathrow Airport                  88809.0
Orlando International Airport            70930.0
Los Angeles International Airport        70530.0
McCarran International Airport           69630.0
John F. Kennedy International Airport    66270.0
Paris Charles de Gaulle Airport          64490.0
Suvarnabhumi Airport                     59460.0
Miami International Airport              58150.0
San Francisco International Airport      58000.0
Dubai International Airport              55590.0
Name: pax, dtype: float64